# Synchrophasor SVD-Based Real-Time Compression

**Notebook purpose:**

- Implement the paper *"A Real-Time Synchrophasor Data Compression Method Using Singular Value Decomposition"*.
- Provide code, mathematical interpretation, inline explanations, and diagnostic plots.

**Notebook structure:**

1. Imports and configuration
2. Data loading (magnitude + angle → complex phasor)
3. PMU measurement uncertainty (ε) — how to obtain/estimate
4. Reduced SVD (thin SVD) and per-mode SNR-based selection (paper's Algorithm 1)
5. Sliding-window rank tracking
6. Progressive partitioning (real-time state machine)
7. Partition compression, reconstruction, and metrics (CR, RMSE, MADE)
8. Visualizations and interpretation

**Notes:** The notebook uses deterministic (thin) SVD (`np.linalg.svd(..., full_matrices=False)`) as in the paper. It treats ε as the PMU measurement RMS error bound and uses the criterion σ_r > ε * sqrt(h n) to select modes.


In [ ]:
# 1) Imports and utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import svd
import math
import os

# plotting defaults
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['font.size'] = 12

def mag_ang_to_complex(mag, ang_deg):
    """Convert magnitude (pu) and angle (degrees) to complex phasor."""
    ang_rad = np.deg2rad(ang_deg)
    return mag * (np.cos(ang_rad) + 1j * np.sin(ang_rad))

def ensure_2d(a):
    arr = np.array(a)
    if arr.ndim == 1:
        return arr[:, None]
    return arr


## 2) Data loading

Load your magnitude and angle CSV files (wide format):

- Columns: `Time, V2_mag, V3_mag, ..., V2_ang, V3_ang, ...` or separate files for mag and ang.

**Important:** the notebook expects **aligned timestamps** across magnitude and angle. If timestamps differ, resample or interpolate before forming complex phasors.


In [ ]:
# Example code to load magnitude and angle CSV files (edit paths to your files)
mag_path = 'magnitude_fault.csv'   # replace with your path or uploaded file
ang_path = 'angle_fault.csv'       # replace with your path or uploaded file

if os.path.exists(mag_path) and os.path.exists(ang_path):
    mag_df = pd.read_csv(mag_path)
    ang_df = pd.read_csv(ang_path)
    # assume the first column is Time
    time = mag_df.iloc[:,0].values
    mag = mag_df.iloc[:,1:].values
    ang = ang_df.iloc[:,1:].values
    # ensure shapes align
    assert mag.shape == ang.shape, "Magnitude and angle matrices must match shape."
    print('Loaded magnitude and angle with shape', mag.shape)
else:
    print('Place your magnitude and angle CSV files in the notebook folder or edit the paths.\n'
          'This notebook will still write out the full code; you can run cells after placing your files.')


## 3) PMU measurement uncertainty (ε)

Paper's approach: do not iterate; use PMU accuracy bound ε to reject modes dominated by measurement error.

**Options to obtain ε:**

1. **From PMU datasheet or IEEE C37.118:** use TVE or RMS error in p.u. (typical magnitudes: 1e-4 p.u. for magnitude, 1e-3 rad for angle). 
2. **Estimate from quiet window:** select a steady interval and compute RMS of residuals.

**Implementation note:** Use the same ε units as the entries of Y (if Y is complex phasor in p.u., ε in p.u.).


In [ ]:
def estimate_epsilon_from_quiet(Y_quiet):
    """Estimate epsilon (RMS) from a quiet segment Y_quiet (h_q x n).
    Returns scalar eps (RMS of residuals).
    """
    Yq = np.array(Y_quiet, dtype=np.complex128)
    # remove per-channel mean to isolate noise
    residuals = Yq - np.mean(Yq, axis=0, keepdims=True)
    eps_est = np.sqrt(np.mean(np.abs(residuals)**2))
    return eps_est

print('Estimate epsilon function ready.')


## 4) Reduced SVD (RSVD in the paper = reduced thin SVD) and mode selection (Algorithm 1)

Mathematical summary:

Given windowed matrix \(Y \in \mathbb{C}^{h\times n}\):

\(Y = U \Sigma V^H\), with singular values \(\sigma_1\ge ... \ge \sigma_m\) (m=min(h,n)).

RMS contribution of mode r: 

\(Y_{r,RMS} = \sigma_r / \sqrt{h n}\).

Measurement RMS bound: \(E_{RMS} \le \epsilon\).

Mode keep criterion (paper eq. 11):

\(\sigma_r > \epsilon \sqrt{h n}\). 

Therefore: \(\rho = \max \{ r : \sigma_r > \epsilon\sqrt{hn} \}\).

This is non-iterative and directly uses device accuracy.

Below are functions to compute SVD, compute rho, and return truncated components.


In [ ]:
def compute_rho_and_components(Y, eps):
    """Compute thin SVD and select rho using the paper's criterion.
    Returns rho, Ur, Sr (vector), Vhr.
    """
    Y = np.array(Y, dtype=np.complex128)
    h, n = Y.shape
    # thin SVD
    U, S, Vh = np.linalg.svd(Y, full_matrices=False)
    # threshold
    threshold = eps * math.sqrt(h * n)
    rho = int(np.sum(S > threshold))
    rho = max(1, rho)
    Ur = U[:, :rho]
    Sr = S[:rho]
    Vhr = Vh[:rho, :]
    return rho, Ur, Sr, Vhr

# small self-check with a random low-rank matrix
h_test, n_test, r_test = 50, 10, 3
A = np.random.randn(h_test, r_test) @ np.random.randn(r_test, n_test)
# choose eps very small so all modes kept
rho_test, U_test, S_test, Vh_test = compute_rho_and_components(A, eps=1e-12)
print('Self-check rho (should be >= r_test):', rho_test)


## 5) Sliding-window rank tracking (ρ(t))

Compute ρ for each sliding window to obtain a time series ρ(t). This series is used by the progressive partitioning algorithm to detect events.

Paper parameters:
- Sampling rate: 60 Hz
- Window size h: 200 (paper choice)


In [ ]:
def sliding_rho_time_series(Y_full, h, eps, step=1):
    """Compute rho(t) for sliding windows over Y_full (T x n) using windows of height h.
       Returns rho_series (length ~ (T-h+1)).
    """
    T, n = Y_full.shape
    rhos = []
    times = []
    for start in range(0, T - h + 1, step):
        Yw = Y_full[start:start+h, :]
        rho, _, _, _ = compute_rho_and_components(Yw, eps)
        rhos.append(rho)
        times.append(start)
    return np.array(times), np.array(rhos)

print('Sliding rho function ready.')


## 6) Progressive partitioning (real-time state machine)

The partitioning logic from the paper (simplified):

- Maintain a buffer that accumulates windows. Monitor ρ(t).
- If ρ increases, start a disturbance partition (φ=1).
- While disturbance continues (ρ_avg >= α ρ_max), keep adding.
- If ρ_avg < α ρ_max, end the disturbance partition (φ transitions) and compress.

Paper settings: α=0.5, l = fs (used as stability counter), h minimum partition size is typically the window size or a multiple.


In [ ]:
def progressive_partition_and_compress(Y_full, h, eps, alpha=0.5, min_partition_len=None):
    """Perform progressive partitioning on Y_full and compress each partition using paper's SVD criterion.
       Returns a list of partition metadata and reconstructed data for evaluation.
    """
    if min_partition_len is None:
        min_partition_len = h

    T, n = Y_full.shape
    partitions = []  # list of (start_idx, end_idx, rho, CR, RMSE, MADE, Y_approx)

    # State variables
    phi = 0
    sigma = 0.0
    ctr = 0
    rho_max = 0
    rho_avg = 0

    buffer = np.zeros((0, n), dtype=np.complex128)

    t = 0
    prev_rho = 0
    while t < T:
        # append one timestamp row
        buffer = np.vstack([buffer, Y_full[t:t+1, :]])
        # compute rho on current buffer
        rho_now, Ur, Sr, Vhr = compute_rho_and_components(buffer, eps)

        if phi == 0:
            # normal state
            if rho_now > prev_rho:
                # disturbance begins
                phi = 1
                sigma = rho_now
                ctr = 1
                rho_max = rho_now
                rho_avg = sigma / ctr
            else:
                # remain in normal; if buffer long enough, compress as normal partition
                if buffer.shape[0] >= min_partition_len:
                    Y_approx = Ur @ np.diag(Sr) @ Vhr
                    rho_use = rho_now
                    h_buf = buffer.shape[0]
                    CR_val = (h_buf * n) / (rho_use * (h_buf + n + 1))
                    RMSE_val = np.linalg.norm(buffer - Y_approx, 'fro') / math.sqrt(h_buf * n)
                    MADE_val = np.max(np.abs(buffer - Y_approx))
                    partitions.append((t - h_buf + 1, t + 1, rho_use, CR_val, RMSE_val, MADE_val, Y_approx.copy()))
                    buffer = np.zeros((0, n), dtype=np.complex128)
        elif phi == 1:
            # disturbance accumulating
            sigma += rho_now
            ctr += 1
            rho_max = max(rho_max, rho_now)
            rho_avg = sigma / ctr
            if rho_avg < alpha * rho_max:
                # end of disturbance
                Y_approx = Ur @ np.diag(Sr) @ Vhr
                rho_use = rho_now
                h_buf = buffer.shape[0]
                CR_val = (h_buf * n) / (rho_use * (h_buf + n + 1))
                RMSE_val = np.linalg.norm(buffer - Y_approx, 'fro') / math.sqrt(h_buf * n)
                MADE_val = np.max(np.abs(buffer - Y_approx))
                partitions.append((t - h_buf + 1, t + 1, rho_use, CR_val, RMSE_val, MADE_val, Y_approx.copy()))
                buffer = np.zeros((0, n), dtype=np.complex128)
                phi = 0
                sigma = 0
                ctr = 0
                rho_max = 0
        prev_rho = rho_now
        t += 1
    return partitions

print('Progressive partitioning and compression ready.')


## 7) Reconstruction and evaluation

Formulas recap:

- \(CR = \dfrac{h n}{\rho (h + n + 1)}\)
- \(RMSE = \dfrac{\|Y - \hat{Y}\|_F}{\sqrt{h n}}\)
- \(MADE = \max |Y - \hat{Y}|\)


In [ ]:
def reconstruct_and_metrics(Y, Ur, Sr, Vhr):
    Yhat = Ur @ np.diag(Sr) @ Vhr
    h, n = Y.shape
    rho = Sr.shape[0]
    CR_val = (h * n) / (rho * (h + n + 1))
    RMSE_val = np.linalg.norm(Y - Yhat, 'fro') / math.sqrt(h * n)
    MADE_val = np.max(np.abs(Y - Yhat))
    return Yhat, CR_val, RMSE_val, MADE_val

print('Reconstruct and metrics ready.')


## 8) Visualization helpers

- Plot rank series
- Plot original vs reconstructed for a chosen channel and partition


In [ ]:
def plot_rank_series(times, rhos, partitions=None):
    plt.figure()
    plt.plot(times, rhos, '-o', label='rho')
    if partitions:
        for (s,e, *_ ) in partitions:
            plt.axvspan(s, e, color='orange', alpha=0.15)
    plt.xlabel('window start index')
    plt.ylabel('rho')
    plt.title('Sliding-window estimated rank (rho)')
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_partition_original_vs_recon(Y_full, partition, channel_index=0):
    s, e, rho, CR_val, RMSE_val, MADE_val, Y_approx = partition
    orig = Y_full[s:e, channel_index]
    recon = Y_approx[:, channel_index]
    t = np.arange(s, e)
    plt.figure()
    plt.plot(t, orig.real, label='orig (real part)')
    plt.plot(t, recon.real, '--', label='recon (real part)')
    plt.xlabel('sample index')
    plt.ylabel('value')
    plt.title(f'Partition {s}-{e}, rho={rho}, CR={CR_val:.2f}, RMSE={RMSE_val:.3e}')
    plt.legend()
    plt.grid(True)
    plt.show()

print('Plot helpers ready.')


---

### How to run this notebook

1. Place your magnitude and angle CSVs (or a pre-built complex phasor CSV) in the folder.
2. Edit file paths in the Data Loading cell.
3. Choose `h = 200` and specify `eps` from PMU spec or estimate a quiet window using the provided function.
4. Execute the sliding rho computation, then progressive partitioning and compression.

If you want, tell me now to run this notebook on the files you previously uploaded; I can execute the analysis and return output tables and figures.
